# Modeling of Walmart Data (2015)

## Date: 12/30/2017
## Author: Hiro Miyake

# Introduction

This notebook performs modeling on data from the [Walmart Recruiting: Trip Type Classification](https://www.kaggle.com/c/walmart-recruiting-trip-type-classification) competition on Kaggle, held in 2015.

# Importation of modules and data

First load the relevant modules.

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import re

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

from xgboost.sklearn import XGBClassifier

In [3]:
## Check the local version of pandas
#pd.__version__

## Check the local version of sklearn
#import sklearn
#sklearn.__version__

Then load the data.

In [4]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

Check the shape of the data.

In [5]:
## If the pandas dataframe has size N x M, df.size gives the value of N*M
## On the other hand, the output of df.shape is a tuple (N, M)
print df_train.size
print df_train.shape
print df_train.shape[0]*df_train.shape[1]
print df_test.size
print df_test.shape

4529378
(647054, 7)
4529378
3921876
(653646, 6)


Check the first 5 lines of the training and testing data.

In [6]:
df_train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0


In [7]:
df_test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0


Concatenate the training and test data vertically. Leave out the target variable (`TripType`) from the train data.

In [8]:
df_all = pd.concat([df_train.iloc[:,1:], df_test], axis = 0)
df_all.iloc[df_train.shape[0],:]

VisitNumber                        1
Weekday                       Friday
Upc                      7.25034e+10
ScanCount                          1
DepartmentDescription          SHOES
FinelineNumber                  3002
Name: 0, dtype: object

# One-Hot-Encoding of `Weekday` Variable

Convert the variable `Weekday` into one-hot-encoded variable. Note you can use `OneHotEncoder` on integers only, i.e., not strings! Need to do `LabelEncoding` first. I do this here with the pandas `get_dummies` function. Once the dummy variables are concatenated to the original data set, I drop the original `Weekday` column.

In [9]:
## With pandas and scikit-learn
## https://chrisalbon.com/machine-learning/one-hot_encode_nominal_categorical_features.html
## https://chrisalbon.com/machine-learning/one-hot_encode_features_with_multiple_labels.html
## https://chrisalbon.com/machine-learning/preprocessing_categorical_features.html
## http://pbpython.com/categorical-encoding.html

## With scikit-learn
## http://www.ritchieng.com/machinelearning-one-hot-encoding/

df_all2 = df_all.copy()

## https://chrisalbon.com/machine-learning/preprocessing_categorical_features.html
df_weekday = pd.get_dummies(df_all2["Weekday"])

## http://pandas.pydata.org/pandas-docs/stable/merging.html
## https://chrisalbon.com/python/pandas_join_merge_dataframe.html
df_all2 = pd.concat([df_all2, df_weekday], axis=1)

## https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.drop.html
df_all2.drop(["Weekday","Upc","ScanCount","DepartmentDescription","FinelineNumber"], axis = 1, inplace = True)
df_all2.drop_duplicates(subset = "VisitNumber", inplace = True)
print df_all2.head()

    VisitNumber  Friday  Monday  Saturday  Sunday  Thursday  Tuesday  \
0             5       1       0         0       0         0        0   
1             7       1       0         0       0         0        0   
3             8       1       0         0       0         0        0   
26            9       1       0         0       0         0        0   
29           10       1       0         0       0         0        0   

    Wednesday  
0           0  
1           0  
3           0  
26          0  
29          0  


# One-Hot-Encoding of `DepartmentDescription` Variable

Once the dummy variables are concatenated horizontally to the original dataframe, I drop the original `DepartmentDescription` column.

In [10]:
df_dd = pd.get_dummies(df_all["DepartmentDescription"])
df_all3 = pd.concat([df_all, df_dd], axis=1)
df_all3.drop(["DepartmentDescription"], axis = 1, inplace = True)
df_all3.drop(["Weekday","Upc","FinelineNumber"], axis = 1, inplace = True)
#df_all3.drop(["Weekday","Upc","ScanCount","FinelineNumber"], axis = 1, inplace = True)
print df_all3.head()

   VisitNumber  ScanCount  1-HR PHOTO  ACCESSORIES  AUTOMOTIVE  BAKERY  \
0            5         -1           0            0           0       0   
1            7          1           0            0           0       0   
2            7          1           0            0           0       0   
3            8          2           0            0           0       0   
4            8          2           0            0           0       0   

   BATH AND SHOWER  BEAUTY  BEDDING  BOOKS AND MAGAZINES    ...     SEAFOOD  \
0                0       0        0                    0    ...           0   
1                0       0        0                    0    ...           0   
2                0       0        0                    0    ...           0   
3                0       0        0                    0    ...           0   
4                0       0        0                    0    ...           0   

   SEASONAL  SERVICE DELI  SHEER HOSIERY  SHOES  SLEEPWEAR/FOUNDATIONS  \
0     

Now I put the `ScanCount` value in the corresponding dummy variable for the `DepartmentDescription`.

In [11]:
#df_all3.head()
df_all3_2 = df_all3.copy()
for i in df_all3_2:
    if i not in ["VisitNumber", "ScanCount"]:
        df_all3_2[i] = df_all3_2["ScanCount"]*df_all3_2[i]

df_all3_2.head()

,VisitNumber,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,5,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Then I aggregate the values based on the `VisitNumber` so that there is one row for each `VisitNumber`.

In [12]:
## https://stackoverflow.com/questions/19384532/how-to-count-number-of-rows-in-a-group-in-pandas-group-by-object
## https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
#df[['col1', 'col2', 'col3', 'col4']].groupby(['col1', 'col2']).agg(['mean', 'count'])
#df5.groupby(['TripType', 'VisitNumber']).agg(['sum'])

df_all4 = df_all3_2.groupby('VisitNumber',as_index = False).agg('sum')
print df_all4.head()

   VisitNumber  ScanCount  1-HR PHOTO  ACCESSORIES  AUTOMOTIVE  BAKERY  \
0            1          4           0            0           0       0   
1            2          4           0            0           0       1   
2            3          0           0            0           0       0   
3            4          1           0            0           0       0   
4            5         -1           0            0           0       0   

   BATH AND SHOWER  BEAUTY  BEDDING  BOOKS AND MAGAZINES    ...     SEAFOOD  \
0                0       0        0                    0    ...           0   
1                0       0        0                    0    ...           0   
2                0       0        0                    0    ...           0   
3                0       0        0                    0    ...           0   
4                0       0        0                    0    ...           0   

   SEASONAL  SERVICE DELI  SHEER HOSIERY  SHOES  SLEEPWEAR/FOUNDATIONS  \
0     

# Make `Return` column

It's possible returns (i.e., negative `ScanCount` values) are fundamentally different from purchases. So I make a separate binary variable called `Return` which is 1 if there was a return and 0 otherwise. This is a flag to indicate if a visit included a return or not.

In [13]:
df_all_return = df_all.copy()
df_all_return = df_all_return[["VisitNumber", "ScanCount"]]
#print df_all_return.head()
df_all_return["Return"] = df_all_return["ScanCount"] < 0
#print df_all_return.head()
df_all_return = df_all_return.groupby('VisitNumber',as_index = False).agg('sum')
#print df_all_return.head()
df_all_return["Return"] = df_all_return["Return"] > 0
#print df_all_return.head()
df_all_return["Return"] = df_all_return["Return"].astype("int")
#print df_all_return.describe()
df_all_return.drop(["ScanCount"], axis = 1, inplace = True)
print df_all_return.head()

   VisitNumber  Return
0            1       0
1            2       0
2            3       1
3            4       0
4            5       1


# One-hot-encoding of `FinelineNumber` variable

There are 9324 distinct `FinelineNumber` values (at least in the train set), which may be too large to easily process. We can try to take just the most frequently occuring values instead.

In [14]:
df_fn_vc = df_all["FinelineNumber"].value_counts()
thresh = 1000
print df_fn_vc[df_fn_vc > thresh].shape
df_fn_vc = df_fn_vc[df_fn_vc > thresh]
#print df_fn[df_fn > thresh]

(272,)


We see that there are 272 distinct `FinelineNumber` counts that are above 1000.

Let's make columns with each `FinelineNumber` with count above 1000.

In [15]:
#df_fn_vc.keys()
df_fn = df_all.copy()
for i in df_fn_vc.keys():
    df_fn[str(i)] = 0

df_fn.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,5501.0,1508.0,135.0,808.0,...,4408.0,1014.0,8251.0,1352.0,9550.0,1510.0,2352.0,9114.0,2804.0,7557.0
0,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Friday,6.053882e+10,1,SHOES,8931.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_fn.drop(["Weekday", "Upc", "DepartmentDescription"], axis = 1, inplace = True)

In [17]:
print df_fn.shape
print df_fn.head()

(1300700, 275)
   VisitNumber  ScanCount  FinelineNumber  5501.0  1508.0  135.0  808.0  0.0  \
0            5         -1          1000.0       0       0      0      0    0   
1            7          1          8931.0       0       0      0      0    0   
2            7          1          4504.0       0       0      0      0    0   
3            8          2          3565.0       0       0      0      0    0   
4            8          2          1017.0       0       0      0      0    0   

   9546.0  1407.0   ...    4408.0  1014.0  8251.0  1352.0  9550.0  1510.0  \
0       0       0   ...         0       0       0       0       0       0   
1       0       0   ...         0       0       0       0       0       0   
2       0       0   ...         0       0       0       0       0       0   
3       0       0   ...         0       0       0       0       0       0   
4       0       0   ...         0       0       0       0       0       0   

   2352.0  9114.0  2804.0  7557.0  
0    

Now I do something similar to what I did with `DepartmentDescription`, which is to give each dummy variable for `FinelineNumber` the `ScanCount` value. In order to have a routine that runs in $O(N)$ time, I make a dictionary of the chosen `FinelineNumber` value and it's column position. Then I can iterate row-wise looking at the `FinelineNumber` value and looking up the corresponding column position with the dictionary and assigning the value from `ScanCount` there. Using a dictionary also makes it very easy to check if the `FinelineNumber` in question is one of the top values. This stems from the fact that dictionaries in Python are hash tables.

In [18]:
#df_fn.head()
nrow = df_fn.shape[0]
ncol = df_fn.shape[1]
#print df_fn.columns.values[1]
#df_fn.iloc[1, 1]
#fn_top = set(df_fn.columns.values)

df_fn2 = df_fn.copy()

fn_top_dict = {}
for i, j in enumerate(df_fn2.columns.values):
    fn_top_dict[j] = i
#print fn_top_dict
#fn_top_dict['5501.0']
#'5501.0' in fn_top_dict

k = 0
for i in xrange(nrow):
    if str(df_fn2.iat[i, 2]) in fn_top_dict:
        df_fn2.iat[i,fn_top_dict[str(df_fn2.iat[i, 2])]] = df_fn2.iat[i,1]
        if k < 10:
            print str(df_fn2.iat[i, 2]) + ' and ' + str(df_fn2.iat[i,1])
            k += 1
                
print df_fn2.head()

1000.0 and -1
4501.0 and 1
4606.0 and 1
3504.0 and 2
3504.0 and -1
3300.0 and 1
707.0 and 1
1001.0 and 3
115.0 and 1
103.0 and 1
   VisitNumber  ScanCount  FinelineNumber  5501.0  1508.0  135.0  808.0  0.0  \
0            5         -1          1000.0       0       0      0      0    0   
1            7          1          8931.0       0       0      0      0    0   
2            7          1          4504.0       0       0      0      0    0   
3            8          2          3565.0       0       0      0      0    0   
4            8          2          1017.0       0       0      0      0    0   

   9546.0  1407.0   ...    4408.0  1014.0  8251.0  1352.0  9550.0  1510.0  \
0       0       0   ...         0       0       0       0       0       0   
1       0       0   ...         0       0       0       0       0       0   
2       0       0   ...         0       0       0       0       0       0   
3       0       0   ...         0       0       0       0       0       0   
4    

In [19]:
df_fn2.drop(["ScanCount", "FinelineNumber"], axis = 1, inplace = True)
df_fn2 = df_fn2.groupby('VisitNumber',as_index = False).agg('sum')
print df_fn2.head()

   VisitNumber  5501.0  1508.0  135.0  808.0  0.0  9546.0  1407.0  4606.0  \
0            1       0       0      0      0    0       0       0       0   
1            2       0       0      0      0    0       0       0       0   
2            3       0       0      0      0    0       0       0       0   
3            4       0       0      0      0    0       0       0       0   
4            5       0       0      0      0    0       0       0       0   

   115.0   ...    4408.0  1014.0  8251.0  1352.0  9550.0  1510.0  2352.0  \
0      0   ...         0       0       0       0       0       0       0   
1      0   ...         1       0       0       0       0       0       0   
2      0   ...         0       0       0       0       0       0       0   
3      0   ...         0       0       0       0       0       0       0   
4      0   ...         0       0       0       0       0       0       0   

   9114.0  2804.0  7557.0  
0       0       0       0  
1       0       0       

In [20]:
## DO NOT RUN THIS!!!
## Do this if you want to include all distinct FinelineNumber as predictors
## This is probably too many variables for the computer to handle in a reasonable amount of time
#df_fn = pd.get_dummies(df_all["FinelineNumber"])
#df_all_fn = pd.concat([df_all, df_fn], axis=1)
#df_all_fn.drop(["FinelineNumber"], axis = 1, inplace = True)
##df_all_fn.drop(["Weekday","Upc","FinelineNumber"], axis = 1, inplace = True)
##df_all3.drop(["Weekday","Upc","ScanCount","FinelineNumber"], axis = 1, inplace = True)
#print df_all_fn.head()

# Join the different preprocessed dataframes into a single dataframe

Join `Weekday` and `DepartmentDescription` and `Return` on `VisitNumber`.

In [21]:
## https://chrisalbon.com/python/pandas_join_merge_dataframe.html
df_all5 = pd.merge(df_all2, df_all4, on='VisitNumber') ## Add Weekday and DepartmentDescription variables
#print df_all5.head()
df_all5 = pd.merge(df_all5, df_all_return, on='VisitNumber') ## Add Return variable
#print df_all5.head()
df_all5 = pd.merge(df_all5, df_fn2, on = "VisitNumber") ## Add FinelineNumber variables
print df_all5.head()

   VisitNumber  Friday  Monday  Saturday  Sunday  Thursday  Tuesday  \
0            5       1       0         0       0         0        0   
1            7       1       0         0       0         0        0   
2            8       1       0         0       0         0        0   
3            9       1       0         0       0         0        0   
4           10       1       0         0       0         0        0   

   Wednesday  ScanCount  1-HR PHOTO   ...    4408.0  1014.0  8251.0  1352.0  \
0          0         -1           0   ...         0       0       0       0   
1          0          2           0   ...         0       0       0       0   
2          0         28           0   ...         0       0       0       0   
3          0          3           0   ...         0       0       0       0   
4          0          3           0   ...         0       0       0       0   

   9550.0  1510.0  2352.0  9114.0  2804.0  7557.0  
0       0       0       0       0       0     

# Split dataframe back into train and test set

First figure out how many unique `VisitNumber` rows are in the train and test set.

In [22]:
train_row = df_train.drop_duplicates(subset = "VisitNumber").shape[0]
test_row = df_test.drop_duplicates(subset = "VisitNumber").shape[0]
#print train_row + test_row

Then split this back into the train and test set.

In [23]:
#print df_all5.shape
df_train_final = df_all5.iloc[:(train_row - 1),:]
df_test_final = df_all5.iloc[train_row:,:]
df_train_final.head()
#print df_test_final.head()

,VisitNumber,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,ScanCount,1-HR PHOTO,...,4408.0,1014.0,8251.0,1352.0,9550.0,1510.0,2352.0,9114.0,2804.0,7557.0
0,5,1,0,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,8,1,0,0,0,0,0,0,28,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
4,10,1,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0


Add the `TripType` outcome variable back into the preprocessed training data set.

In [24]:
df_train_target = df_train.drop_duplicates(subset = "VisitNumber")
df_train_final2 = pd.merge(df_train_target[["TripType","VisitNumber"]], df_train_final, on = "VisitNumber")
df_train_final2.head()

,TripType,VisitNumber,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,ScanCount,...,4408.0,1014.0,8251.0,1352.0,9550.0,1510.0,2352.0,9114.0,2804.0,7557.0
0,999,5,1,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
1,30,7,1,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,26,8,1,0,0,0,0,0,0,28,...,0,0,0,0,0,0,0,0,0,0
3,8,9,1,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,8,10,1,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


# Train the model

The predictions are to be evaluated using the multi-class logarithmic loss function. This is given as

$$
\begin{equation*}
C = -\frac{1}{N} \sum_{i=1,j=1}^{i=N,j=M} y_{i,j} \log(p_{i,j})
\end{equation*}
$$

where $N$ is the number of visits in the test set, $M$ is the number of trip types, $y_{i,j}$ is 1 if observation $i$ is of class $j$ and 0 otherwise, and $p_{i,j}$ is the predicted probability that observation $i$ belongs to class $j$.

If desired, take a stratified subset of the training data for quicker data processing and iteration.

In [25]:
## See a list of all of the variables in the training model
#df_train_final2.info()

## http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
## https://chrisalbon.com/machine-learning/split_data_into_training_and_test_sets.html
## df_train, df_test = train_test_split(df, test_size=0.33, random_state=100, stratify = df['y'])
df_train_final2_train, df_train_final2_test = train_test_split(df_train_final2, train_size=0.5, 
                                                             random_state=0, stratify = df_train_final2['TripType'])

In [27]:
## Logistic regression
## http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
## http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
## solver liblinear does one-vs-rest classification
## solver saga does true multinomial classification; but I have sklearn version 0.18 which does not have saga
#model = linear_model.LogisticRegression(C = 100) ## vanilla logistic regression with liblinear solver
model = linear_model.LogisticRegression(C = 100, solver = 'sag', multi_class = 'multinomial')

## Random forest
## http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
#model = RandomForestClassifier(max_depth=5, random_state=0)

## XGBoost
## http://xgboost.readthedocs.io/en/latest/python/python_api.html
## http://xgboost.readthedocs.io/en/latest/parameter.html
##xgb = XGBClassifier(max_depth=10, learning_rate=0.1, n_estimators=100,
##                    objective='binary:logistic', subsample=0.6, colsample_bytree=0.6, seed=0)
#model = XGBClassifier(max_depth=6, learning_rate=0.25, n_estimators=43,
#                    objective='binary:logistic', subsample=0.6, colsample_bytree=0.6, seed=0)
#model = XGBClassifier(max_depth=6, learning_rate=0.25, n_estimators=43,
#                    objective='multi:softmax', subsample=0.6, colsample_bytree=0.6, seed=0)

##xgb.fit(X, y)
##xgb.fit(X, y, eval_metric='mlogloss')
#xgb.fit(X, y, eval_metric='logloss')
#y_pred = xgb.predict_proba(X_test) 

#df_train_final3 = df_train_final2_train.drop(["VisitNumber"], axis = 1)
#df_train_final3 = df_train_final2.drop(["VisitNumber"], axis = 1)
#df_train_final3.head()

model.fit(df_train_final2_train.iloc[:, 2:], df_train_final2_train["TripType"])
#model.fit(df_train_final3.iloc[:, 1:], df_train_final3["TripType"])
#model.fit(df_train_final3.iloc[:, 1:], df_train_final3["TripType"], eval_metric='mlogloss')

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

Figure out the log loss value for the training set.

In [28]:
## http://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html
## http://www.markhneedham.com/blog/2016/09/14/scikit-learn-first-steps-with-log_loss/
pred_train = model.predict_proba(df_train_final2_train.iloc[:, 2:])
score = log_loss(df_train_final2_train["TripType"], pred_train)
#pred_train = model.predict_proba(df_train_final3.iloc[:, 1:])
#score = log_loss(df_train_final3["TripType"], pred_train)
print 'Logloss for the training set: ' + str(score)

Logloss for the training set: 1.32400215151


Figure out the log loss value for the held out test set.

In [29]:
pred_train = model.predict_proba(df_train_final2_test.iloc[:, 2:])
score = log_loss(df_train_final2_test["TripType"], pred_train)
print 'Logloss for the validation set: ' + str(score)

Logloss for the validation set: 1.35820605521


# Make predictions on the test set

In [30]:
df_test_final.head()
df_test_final2 = df_test_final.drop(["VisitNumber"], axis = 1)

In [31]:
#x = model.predict(df_test_final2)
x = model.predict_proba(df_test_final2)
#print x
#x2 = pd.DataFrame(x)
#x2.head()
#x2[0].value_counts()

In [32]:
x2 = pd.DataFrame(x)
x2.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.004925,0.005545,0.022998,0.013690,0.059123,0.162303,0.045482,0.013572,0.000464,0.011651,...,0.015379,0.019908,0.192179,0.096153,0.008056,0.016076,0.017241,0.016894,0.011524,0.026146
1,0.018349,0.006171,0.021981,0.017885,0.037760,0.193030,0.051477,0.009875,0.000482,0.031233,...,0.021821,0.011439,0.026365,0.095827,0.005012,0.016065,0.017095,0.023244,0.008508,0.035510
2,0.004345,0.001045,0.006932,0.002170,0.008448,0.057419,0.056938,0.000546,0.000786,0.000715,...,0.001491,0.000573,0.000905,0.001986,0.000062,0.000436,0.000455,0.000577,0.000193,0.836700
3,0.018075,0.005747,0.038207,0.012307,0.039351,0.250995,0.368642,0.003921,0.002529,0.005528,...,0.008422,0.003709,0.005575,0.013847,0.000517,0.003799,0.005260,0.005425,0.002220,0.083982
4,0.004345,0.001045,0.006932,0.002170,0.008448,0.057419,0.056938,0.000546,0.000786,0.000715,...,0.001491,0.000573,0.000905,0.001986,0.000062,0.000436,0.000455,0.000577,0.000193,0.836700


# Create submission csv file from the test set predictions

In [33]:
x2 = pd.DataFrame(x)
x2.head()

## Copy and paste column names from sample submission file
predcols = ["TripType_3","TripType_4","TripType_5","TripType_6",
            "TripType_7","TripType_8","TripType_9","TripType_12","TripType_14",
            "TripType_15","TripType_18","TripType_19","TripType_20","TripType_21",
            "TripType_22","TripType_23","TripType_24","TripType_25","TripType_26",
            "TripType_27","TripType_28","TripType_29","TripType_30","TripType_31",
            "TripType_32","TripType_33","TripType_34","TripType_35","TripType_36",
            "TripType_37","TripType_38","TripType_39","TripType_40","TripType_41",
            "TripType_42","TripType_43","TripType_44","TripType_999"]

x2.columns = predcols
#print x2.head()

d = pd.DataFrame(0, index=np.arange(len(df_test["VisitNumber"].drop_duplicates())), columns=["VisitNumber"])
d["VisitNumber"] = df_test["VisitNumber"].drop_duplicates().values
#print df_test_final["VisitNumber"]
print d.head()

d = pd.concat([d, x2], axis = 1)
#a = pd.concat([a, x2], axis = 1, ignore_index = True)
d.head()
#x2.head()
#a = df_test["VisitNumber"].drop_duplicates() ## get unique visit numbers
#a = a.values        ## turn unique visit series into a list; necessary to avoid problems with next step
#x2["VisitNumber"] = a ## assign visit numbers from test set for prediction

#x2.head()

#a = x2["TripType_3"]
#x2["TripType_3"] = x2["VisitNumber"]
#x2["VisitNumber"] = a
#x2.head()

   VisitNumber
0            1
1            2
2            3
3            4
4            6


,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.004925,0.005545,0.022998,0.013690,0.059123,0.162303,0.045482,0.013572,0.000464,...,0.015379,0.019908,0.192179,0.096153,0.008056,0.016076,0.017241,0.016894,0.011524,0.026146
1,2,0.018349,0.006171,0.021981,0.017885,0.037760,0.193030,0.051477,0.009875,0.000482,...,0.021821,0.011439,0.026365,0.095827,0.005012,0.016065,0.017095,0.023244,0.008508,0.035510
2,3,0.004345,0.001045,0.006932,0.002170,0.008448,0.057419,0.056938,0.000546,0.000786,...,0.001491,0.000573,0.000905,0.001986,0.000062,0.000436,0.000455,0.000577,0.000193,0.836700
3,4,0.018075,0.005747,0.038207,0.012307,0.039351,0.250995,0.368642,0.003921,0.002529,...,0.008422,0.003709,0.005575,0.013847,0.000517,0.003799,0.005260,0.005425,0.002220,0.083982
4,6,0.004345,0.001045,0.006932,0.002170,0.008448,0.057419,0.056938,0.000546,0.000786,...,0.001491,0.000573,0.000905,0.001986,0.000062,0.000436,0.000455,0.000577,0.000193,0.836700


In [34]:
'''
##*******************************************************************************************
## THIS BLOCK IS NOT NECESSARY ANYMORE!!!
## Was needed to put 1 in the most likely TripType prediction table necessary for submission
##*******************************************************************************************
## Copy and paste column names from sample submission file
predcols = ["VisitNumber","TripType_3","TripType_4","TripType_5","TripType_6",
            "TripType_7","TripType_8","TripType_9","TripType_12","TripType_14",
            "TripType_15","TripType_18","TripType_19","TripType_20","TripType_21",
            "TripType_22","TripType_23","TripType_24","TripType_25","TripType_26",
            "TripType_27","TripType_28","TripType_29","TripType_30","TripType_31",
            "TripType_32","TripType_33","TripType_34","TripType_35","TripType_36",
            "TripType_37","TripType_38","TripType_39","TripType_40","TripType_41",
            "TripType_42","TripType_43","TripType_44","TripType_999"]

## Make a dictionary of trip type number to index
## https://stackoverflow.com/questions/4289331/python-extract-numbers-from-a-string
j = 0
triptypedict = {}
for i in predcols:
    a = re.findall(r'\d+', i)
    #re.findall(r'\d+', 'hello 42 I\'m a 32 string 30')
    if a:
        a = int(a[0])
        triptypedict[a] = j
        j += 1

print triptypedict

## Makes dataframe with correct column names but empty otherwise
## https://stackoverflow.com/questions/44513738/pandas-create-empty-dataframe-with-only-column-names
#d = pd.DataFrame(columns=predcols)

## Make a dataframe with all zeros of the right size
## https://stackoverflow.com/questions/22963263/creating-a-zero-filled-pandas-data-frame
d = pd.DataFrame(0, index=np.arange(len(df_test["VisitNumber"].drop_duplicates())), columns=predcols)
a = df_test["VisitNumber"].drop_duplicates() ## get unique visit numbers
a = a.values        ## turn unique visit series into a list; necessary to avoid problems with next step
d["VisitNumber"] = a ## assign visit numbers from test set for prediction
#print d.head()

## Go through predictions and assign a 1 to the appropriate column
## https://chrisalbon.com/python/pandas_indexing_selecting.html
#print y.iloc[1,0] ## access elements in dataframe by indices
for counter, value in enumerate(x):
    d.iloc[counter, (triptypedict[value] + 1)] = 1
print d.head()
'''

'\n##*******************************************************************************************\n## THIS BLOCK IS NOT NECESSARY ANYMORE!!!\n## Was needed to put 1 in the most likely TripType prediction table necessary for submission\n##*******************************************************************************************\n## Copy and paste column names from sample submission file\npredcols = ["VisitNumber","TripType_3","TripType_4","TripType_5","TripType_6",\n            "TripType_7","TripType_8","TripType_9","TripType_12","TripType_14",\n            "TripType_15","TripType_18","TripType_19","TripType_20","TripType_21",\n            "TripType_22","TripType_23","TripType_24","TripType_25","TripType_26",\n            "TripType_27","TripType_28","TripType_29","TripType_30","TripType_31",\n            "TripType_32","TripType_33","TripType_34","TripType_35","TripType_36",\n            "TripType_37","TripType_38","TripType_39","TripType_40","TripType_41",\n            "TripType_42","Tr

Turn the dataframe into a csv file.

In [35]:
## Don't keep the indices
## https://stackoverflow.com/questions/16923281/pandas-writing-dataframe-to-csv-file
d.to_csv("submit_9.csv", index=False)